# 0D Reactor: Postprocessing the FNN for the Trunk

In [2]:
### Importing Libraries

import sys
print(sys.version)
import os
import time


### Defining WORKSPACE_PATH

# WORKSPACE_PATH = os.environ['WORKSPACE_PATH']
WORKSPACE_PATH = os.path.join(os.getcwd(), '../../../../../../../')
ROMNet_fld     = os.path.join(WORKSPACE_PATH, 'ROMNet/romnet/')

### Importing External Libraries

import numpy                             as np
import pandas                            as pd


### Importing Matplotlib and Its Style

import matplotlib.pyplot                 as plt

#plt.style.use(os.path.join(WORKSPACE_PATH, 'ROMNet/romnet/extra/postprocessing/presentation.mplstyle'))
plt.style.use(os.path.join(WORKSPACE_PATH, 'ROMNet/romnet/extra/postprocessing/zoomed.mplstyle'))
#plt.style.use(os.path.join(WORKSPACE_PATH, 'ROMNet/romnet/extra/postprocessing/paper_1column.mplstyle'))
#plt.style.use(os.path.join(WORKSPACE_PATH, 'ROMNet/romnet/extra/postprocessing/paper_2columns.mplstyle'))


# from scipy.integrate import solve_ivp
# import pyDOE
# from PCAfold         import PCA          as PCAA


import romnet                            as rmnt

3.9.12 (main, Mar 26 2022, 15:51:13) 
[Clang 12.0.0 (clang-1200.0.32.29)]


## Defining Input Variables

In [ ]:
#PCAType         = 'All'

PCAType         = 'OneByOne'
iVar            = 1
Vars            = ['T','H2','H','O','O2','OH','H2O','HO2','H2O2','N','NH','NH2','NH3','NNH','NO','NO2','N2O','HNO','N2']
Var             = Vars[iVar-1]


if (PCAType == 'OneByOne'):
    #path_to_run_fld = os.path.join(WORKSPACE_PATH, 'ROMNet/MSD_100Cases_PCA_'+str(i_red)+'_Trunk/FNN/TestCase3/')
    path_to_run_fld = os.path.join(WORKSPACE_PATH, 'ROMNet/0DReact_Isobaric_500Cases_H2_32PCA/Var'+str(iVar)+'_Trunk/FNN/Run_1/')
elif (PCAType == 'All'):
    path_to_run_fld = os.path.join(WORKSPACE_PATH, 'ROMNet/0DReact_Isobaric_500Cases_H2_32PCA/All_Trunk/FNN/Run_1/')
    #path_to_run_fld = os.path.join(WORKSPACE_PATH, 'ROMNet/MSD_100Cases_PCA_All_Trunk/FNN/Run_1/')

FigDir          = None #os.path.join(WORKSPACE_PATH, '../Desktop/Paper_Figures_DeepONet_TEMP/')


TrainingCases   = [0]#[0,2,4,6,8]
TestCases       = [0,2,4]#[0,2]

LineVec         = ['-',':','--','.-']*10
ColorVec        = ['#190707', '#dd3232', '#0065a9', '#348a00','#985396','#f68b69']

## Loading ROMNet's Input Data File

In [ ]:
np.savetxt('./iVar.csv', np.array([[iVar]], dtype=int), fmt='%i')

print("\n[ROMNet]: Reading Input File from: ", path_to_run_fld)
sys.path.insert(0, path_to_run_fld)

print("\n[ROMNet]: Keep Loading Modules and Functions...")
from ROMNet_Input import inputdata

print("\n[ROMNet]: Initializing Input ...")
InputData                 = inputdata(WORKSPACE_PATH)


InputData.InputFilePath   = path_to_run_fld+'/ROMNet_Input.py'
InputData.train_int_flg   = 0
InputData.path_to_run_fld = path_to_run_fld

## Loading Tensorflow's Model for the Network 

In [ ]:
surrogate_type = InputData.surrogate_type
if (surrogate_type == 'FNN-SourceTerms'):
    surrogate_type = 'FNN'

Net   = getattr(rmnt.architecture, surrogate_type)

model = rmnt.model.Model_TF(InputData)

if (InputData.phys_system is not None):
    System = getattr(rmnt.pinn.system, InputData.phys_system)
    system = System(InputData)
    
model.build(InputData, None, Net, system)#, loadfile_no='000027')

NN    = model.net

## Evaluating Trunk on Test Data

In [ ]:
Vars      = ['t_'+str(i_mode+1) for i_mode in range(InputData.NRODs)]

FileName  = InputData.path_to_data_fld + '/test/pts/Input.csv'# + InputData.InputFile
Data      = pd.read_csv(FileName, header=0)
tVec      = Data['t'].to_numpy()[...,np.newaxis]
VarNames  = list(Data.columns).remove('t')

FileName  = InputData.path_to_data_fld + '/test/pts/Output.csv'# + InputData.InputFile
Data      = pd.read_csv(FileName, header=0)
Output    = Data[Vars].to_numpy()


        
yMat      = model.predict(tVec)    



fig = plt.figure()
#for i in range(InputData.NRODs):
for i in [30]:
    if (i==0):
        Label1 = 'ROM'
        Label2 = 'FNN'
    else:
        Label1 = None
        Label2 = None
    plt.plot(tVec, Output[:,i], '-',   color=ColorVec[0], label=Label1)
    plt.plot(tVec, yMat[:,i],     ':', color=ColorVec[1], label=Label2)

plt.xscale('log')
plt.xlim([1.e-6,1.e0])

plt.xlabel('Time [s]')
plt.ylabel(r'$\phi_{'+Var+'i}$')

plt.legend()

fig.tight_layout()  # otherwise the right y-label is slightly clipped

if (FigDir):
    if (PCAType == 'All'):
        plt.savefig(FigDir+'/0DReact_All_Trunk.eps', format='eps', bbox_inches='tight')
    else:
        plt.savefig(FigDir+'//0DReact_OneByOne_Trunk_'+Var+'.eps', format='eps', bbox_inches='tight')

In [ ]:
print('[PCA]  Max % Error = ', np.max(abs((yMat - Output)/yMat)*100))
print('[PCA]  Max      SE = ', np.max((yMat - Output)**2))
print('[PCA] Mean % Error = ', np.mean(abs((yMat - Output)/yMat)*100))
print('[PCA]          MSE = ', np.mean((yMat - Output)**2))

## Plotting Loss Histories

In [ ]:
Data = pd.read_csv(path_to_run_fld+'/Training/History.csv')

fig  = plt.figure()
plt.plot(Data['tot_loss'],     label='Training')
plt.plot(Data['val_tot_loss'], label='Validation')
plt.xlabel('Epoch')
plt.ylabel('Tot. Loss [MSE]')
plt.legend()
plt.yscale('log')

fig  = plt.figure()
plt.plot(Data['pts_loss'],     label='Training')
plt.plot(Data['val_pts_loss'], label='Validation')
plt.xlabel('Epoch')
plt.ylabel('Data Loss [MSE]')
plt.legend()
plt.yscale('log')
#plt.ylim([1.e-2, 1.e0])

# fig  = plt.figure(figsize=(12,8))
# plt.plot(Data['scs_loss'],     label='Training')
# plt.plot(Data['val_scs_loss'], label='Validation')
# plt.xlabel('Epoch')
# plt.ylabel('S.C.s Loss [MSE]')
# plt.legend()
# plt.yscale('log')

# fig  = plt.figure(figsize=(12,8))
# plt.plot(Data['ics_loss'],     label='Training')
# plt.plot(Data['val_ics_loss'], label='Validation')
# plt.xlabel('Epoch')
# plt.ylabel('I.C.s Loss [MSE]')
# plt.legend()
# plt.yscale('log')

In [ ]:
#path_to_data_fld = InputData.path_to_data_fld
path_to_data_fld = '/Users/sventur/WORKSPACE/ROMNet/Data/0DReact_Isobaric_500Cases_H2/32PCA/OneByOne/Var19/Trunk/'

In [ ]:
tVars           = ['t_'+str(i+1) for i in range(32)]
bVars           = ['b_'+str(i+1) for i in range(32)]

InputDF         = pd.read_csv(path_to_data_fld+'/test/pts/Input.csv')
OutputDF        = pd.read_csv(path_to_data_fld+'/test/pts/Output.csv')

InputDF_branch  = pd.read_csv(path_to_data_fld+'/../Branch/test/pts/Input.csv')
OutputDF_branch = pd.read_csv(path_to_data_fld+'/../Branch/test/pts/Output.csv')

In [ ]:
A               = OutputDF_branch[bVars].to_numpy()
Psi             = OutputDF[tVars].to_numpy()
C               = OutputDF_branch['c'].to_numpy()
D               = OutputDF_branch['d'].to_numpy()

n_scenarios     = A.shape[0]
n_t             = Psi.shape[0]

Y               = np.zeros((n_t,n_scenarios))
for i_scenario in range(n_scenarios):
    Y[:,i_scenario] = np.sum(Psi * A[i_scenario,:], axis=1) * D[i_scenario] + C[i_scenario]

In [ ]:
# plt.plot(InputDF['t'], Psi[:,31])
# plt.xscale('log')

plt.plot(InputDF['t'], Y[:,0])
plt.xscale('log')